In [8]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import load_data, get_label_dict, get_word_labels

from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

Loading train data

Finished loading train data

Loading val data

Finished loading val data

Loading test data

Finished loading test data



In [10]:
labels = get_label_dict()
words = get_word_labels()

test_images = os.listdir("data/tiny-imagenet-200/test/images/")
assert len(X_test) == len(test_images)

Loading labels

Done

Loading words

Done



In [11]:
# Image preprocessing for the data
datagen = ImageDataGenerator(rotation_range=30, 
                             brightness_range=[0.3, 0.7], 
                             shear_range=0.2, 
                             zoom_range=0.2, 
                             horizontal_flip=True, 
                             width_shift_range=0.2,
                             height_shift_range=0.2, 
                             channel_shift_range=90., 
                             fill_mode="reflect", 
                             preprocessing_function=shuffle_channels, 
                             rescale=1./255)

testgen = ImageDataGenerator(rescale=1./255)

data = {"train": (X_train, y_train), 
        "val": (X_val, y_val)}

In [12]:
model = BravoNet()
model_prefix = "models/" + model.name + "/" + model.name
model_path = model_prefix + ".h5"

Model: "BravoNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 2048)              23564800  
_________________________________________________________________
dense_1 (Dense)              (None, 200)               409800    
Total params: 23,974,600
Trainable params: 23,929,160
Non-trainable params: 45,440
_________________________________________________________________


In [13]:
# Checkpoint callback to save best model between epochs
checkpoint = ModelCheckpoint(model_path, 
                             monitor="val_accuracy", 
                             verbose=1, 
                             save_best_only=True, 
                             mode="auto", 
                             period=1)

cb_list = [checkpoint]

In [ ]:
optim = optimizers.Adam(learning_rate=0.001)

history = train(model_path=model_path, 
                restore=False, 
                epochs=200,
                model=model, 
                optim=optim,
                datagen=datagen,
                data=data, 
                cb_list=cb_list)

Epoch 1/200
3125/3125 [==============================] - 270s 86ms/step - loss: 4.8346 - accuracy: 0.0397 - top3_accuracy: 0.0954 - top5_accuracy: 0.1400 - val_loss: 5.4091 - val_accuracy: 0.0494 - val_top3_accuracy: 0.1207 - val_top5_accuracy: 0.1758

Epoch 00001: val_accuracy improved from -inf to 0.04940, saving model to models/BravoNet/BravoNet.h5
Epoch 2/200
3125/3125 [==============================] - 259s 83ms/step - loss: 4.2613 - accuracy: 0.0945 - top3_accuracy: 0.2019 - top5_accuracy: 0.2767 - val_loss: 4.7272 - val_accuracy: 0.0984 - val_top3_accuracy: 0.2103 - val_top5_accuracy: 0.2881

Epoch 00002: val_accuracy improved from 0.04940 to 0.09840, saving model to models/BravoNet/BravoNet.h5
Epoch 3/200
3125/3125 [==============================] - 257s 82ms/step - loss: 3.9589 - accuracy: 0.1377 - top3_accuracy: 0.2696 - top5_accuracy: 0.3553 - val_loss: 4.3959 - val_accuracy: 0.1197 - val_top3_accuracy: 0.2380 - val_top5_accuracy: 0.3120

Epoch 00003: val_accuracy improved f

In [ ]:
show_history(history)

In [ ]:
# This loads the model and makes predictions on the test set
custom_metrics = {
    "top3_accuracy": top3_acc,
    "top5_accuracy": top5_acc
}
model = load_model(model_path, custom_objects=custom_metrics)
output = model.predict_generator(testgen.flow(X_test, shuffle=False))
results = []

for result in output:
    results.append(labels[np.argmax(result)])
    
assert len(results) == len(X_test)
print("Test images predicted:", len(results))

In [ ]:
# This writes the test set predictions to file
with open(model_prefix + " Predictions.txt", "w") as test_file:
    for i in range(len(results)):
        test_file.write(test_images[i] + " " + results[i] + "\n")
    print("Predictions saved at", test_file.name)

In [ ]:
# This shows some test set images and the model's predictions on them
display_dim = 128
for _ in range(10):
    index = np.random.randint(len(X_test))
    img = Image.fromarray(X_test[index], "RGB").resize(size=(display_dim, display_dim))
    word_label = words[results[index]]
    display(img)
    print(word_label)

In [ ]:
# This saves an HTML version of the notebook for later viewing
os.system("%notebook 'Training'")
os.system("jupyter nbconvert --to html Training.ipynb")

html_path = model_prefix + ".html"
if path.exists(html_path):
    os.remove(html_path)
os.rename("Training.html", html_path)